# Calculate the ddG of making each mutation at position 501 in the Spike protein, for the Spike-Ace2 Complex

(Using Chris Mathy's ddg_exercise.ipynb as a template)

As always, we start by initializing our PyRosetta session with an `init` call.

In [1]:
import os
import re
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns

import pyrosetta
from pyrosetta import *
pyrosetta.init('''
               -ex1
               -ex2
               -score:weights ref2015_cart
               -use_input_sc
               -ignore_unrecognized_res           
               ''')
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task import operation
from pyrosetta.rosetta.core.select import residue_selector

scorefxn = create_score_function('ref2015_cart')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.11+release.e9f47978df05d6d2de96236c2d949ecf9436018a 2021-03-19T09:06:15] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r276 2021.11+release.e9f47978df0 e9f47978df05d6d2de96236c2d949ecf9436018a http://www.pyrosetta.org 2021-03-19T09:06:15
core.init: {0} command: PyRosetta -ex1 -ex2 -score:weights ref2015_cart -use_input_sc -ignore_unrecognized_res -database /opt/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=1169127495 seed_offset=0 real_seed=1169127495 thread_index=0
basic.random.init_random_generator: {0} RandomGene

### Make a Pose from our PDB

Kevin made a 'clean' version of the PDB file containing the WT Spike bound to Ace2. Located in: ```Project1-219/Kevin/Spike_ace2.pdb```  
Kevin renamed this 'clean' version to 6M17_clean.pdb.  
The full PDB file is 6M17.

In [2]:
spike_ace2_clean = '../Kevin/6M17_clean.pdb'

pose = pose_from_pdb(spike_ace2_clean)
pose.pdb_info().name('6M17_clean')

core.import_pose.import_pose: {0} File '../Kevin/6M17_clean.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 748
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue LEU:CtermProteinFull 931
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 510 522
core.conformation.Conformation: {0} current variant for 510 CYS
core.conformation.Conformation: {0} current variant for 522 CYS
core.conformation.Conformation: {0} current variant for 510 CYD
core.conformation.Conformation: {0} current variant for 522 CYD
core.confo

### Relax our Pose

Copied code from "relax_1L63.py" in ddg_exercise to "relax_Spike_Ace2.py" and changed the path to the pdb file.  
Should output a pdb, "Spike_Ace2_relaxed.pdb" --> Renamed to "6M17_relaxed.pdb"

In [3]:
spike_ace2_relaxed = './6M17_relaxed.pdb'

pose_relaxed = pose_from_pdb(spike_ace2_relaxed)
pose_relaxed.pdb_info().name('6M17_relaxed')

core.import_pose.import_pose: {0} File './6M17_relaxed.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} Found disulfide between residues 324 341
core.conformation.Conformation: {0} current variant for 324 CYS
core.conformation.Conformation: {0} current variant for 341 CYS
core.conformation.Conformation: {0} current variant for 324 CYD
core.conformation.Conformation: {0} current variant for 341 CYD
core.conformation.Conformation: {0} Found disulfide between residues 792 845
core.conformation.Conformation: {0} current variant for 792 CYS
core.conformation.Conformation: {0} current variant for 845 CYS
core.conformation.Conformation: {0} current variant for 792 CYD
core.conformation.Conformation: {0} current variant for 845 CYD
core.conformation.Conformation: {0} Found disulfide between residues 893 901
core.conformation.Conformation: {0} current variant for 893 CYS
core.conformation.Conformation: {0} current variant for 901 CYS
core.conformation.Conforma

In [4]:
pmm = PyMOLMover()
pmm.apply(pose_relaxed)

In PyMOL, copy and paste the following commands to show the core packing around Asn 501.

`util.cbaw 6M17_relaxed`

`select core, 6M17_relaxed and br. (sc.  within 3 of (resi 501 and chain F and sc.))`

`util.cbac core`

`util.cbas core and resi 501 and chain F`

`show spheres, core`

`set_view (\
     0.749474406,   -0.659760594,   -0.054414708,\
    -0.335082650,   -0.448958933,    0.828328848,\
    -0.570924938,   -0.602581918,   -0.557566702,\
     0.000094488,    0.001106668,  -85.698837280,\
    30.798748016,    4.481562614,    6.916054726,\
    49.036270142,  122.386810303,  -20.000000000 )`

`deselect`

You should see N501 in pink in the center of the frame, well packed in the core of the protein. 

Now run the following commands to show sidechains as sticks instead of as spheres:

`hide spheres, core`

`show sticks, core and not hydro`

It should now be much easier to see how each of the sidechains in the residues shown are pointed inward to form the hydrophobic core of the protein. 

Next, we are going to compute the change in energy for when we mutate N501Y.

In [5]:
three_letter_code = 'ASP'
one_letter_code = 'D'

pose_N501Y = pose_relaxed.clone()
pose_N501Y.pdb_info().name('N501' + one_letter_code)

mutate_mover = pyrosetta.rosetta.protocols.simple_moves.MutateResidue()
mutate_mover.set_target('501F') # Chain F
mutate_mover.set_res_name(three_letter_code)
mutate_mover.apply(pose_N501Y)

In [6]:
# Note: This is how to convert between pdb and rosetta residue numbering

N501_resnum = pose_N501Y.pdb_info().pdb2pose(chain='F',res=501)
N501_resnum
# N501 is residue at index 914 in pyrosetta

914

In [7]:
# score WT and N501Y mutant (before repacking)
WT_score = scorefxn(pose_relaxed)
N501Y_score = scorefxn(pose_N501Y)

print('\nWT score: ' + str(WT_score))
print('\nN501Y score: ' + str(N501Y_score))

core.energy_methods.CartesianBondedEnergy: {0} Creating new peptide-bonded energy container (931)
basic.io.database: {0} Database file opened: scoring/score_functions/elec_cp_reps.dat
core.scoring.elec.util: {0} Read 40 countpair representative atoms
core.pack.dunbrack.RotamerLibrary: {0} shapovalov_lib_fixes_enable option is true.
core.pack.dunbrack.RotamerLibrary: {0} shapovalov_lib::shap_dun10_smooth_level of 1( aka lowest_smooth ) got activated.
core.pack.dunbrack.RotamerLibrary: {0} Binary rotamer library selected: /opt/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin
core.pack.dunbrack.RotamerLibrary: {0} Using Dunbrack library binary file '/opt/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database/rotamer/shapovalov/StpDwn_0-0-0/Dunbrack10.lib.bin'.
core.pack.dunbrack.RotamerLibrary: {0} Dunbrack 2010 library took 0.246496 seconds to load from binary
core.energy_methods.CartesianB

Our Rosetta energy score is ~ 430 (~ 110 kcal/mol) REU worse for the mutant, but this is before relaxing the structure with the mutation.  
  
Let's see what Rosetta energy terms changed:

In [8]:
energy_table_WT = pyrosetta.bindings.energies.residue_total_energies_array(pose_relaxed.energies())
energy_df_WT = pd.DataFrame(energy_table_WT)
totals_by_term_WT = energy_df_WT.sum(axis=0)


energy_table_N501Y = pyrosetta.bindings.energies.residue_total_energies_array(pose_N501Y.energies())
energy_df_N501Y = pd.DataFrame(energy_table_N501Y)
totals_by_term_N501Y = energy_df_N501Y.sum(axis=0)

totals_by_term_N501Y - totals_by_term_WT

fa_atr                 1.636280
fa_rep                -0.352927
fa_sol                 2.457583
fa_intra_rep          -0.199508
fa_intra_sol_xover4    0.050867
lk_ball_wtd            0.131496
fa_elec                1.578500
hbond_sr_bb            0.000000
hbond_lr_bb            0.000000
hbond_bb_sc            0.837584
hbond_sc               0.000000
dslf_fa13              0.000000
omega                  0.000000
fa_dun                -0.260096
p_aa_pp               -0.562633
yhh_planarity          0.000000
ref                   -0.805480
rama_prepro           -0.676571
cart_bonded           -0.428326
total_score            4.653456
dtype: float64

As was seen in the ddg_exercise, the fa_rep term is responsible for the much higher RE score.  This is because we still need to repack the residues at the interface.  Let's repack any residues within 10 Angstroms:  

# Note: I am unable to visualize the N501Y mut in Pymol

In [9]:
# Select mutant position
mutant_position = residue_selector.ResidueIndexSelector()
mutant_position.set_index(914) # N501 is index 914

# Select neighbor positions
nbr_selector = residue_selector.NeighborhoodResidueSelector() 
nbr_selector.set_focus_selector(mutant_position)
nbr_selector.set_include_focus_in_subset(True)
nbr_selector.set_distance(10) # note default is 10, so could leave this out

# Make a task factory
tf = TaskFactory()
tf.push_back(operation.InitializeFromCommandline())
tf.push_back(operation.IncludeCurrent())
tf.push_back(operation.NoRepackDisulfides())

# Disable packing for all residues, then re-enable it for neigbor residues
prevent_repacking_rlt = operation.PreventRepackingRLT()
define_repacking = operation.OperateOnResidueSubset(prevent_repacking_rlt, nbr_selector, True )
tf.push_back(define_repacking)

# Disable design for the whole protein, since we only want to repack
not_design = residue_selector.TrueResidueSelector()
tf.push_back(operation.OperateOnResidueSubset(operation.RestrictToRepackingRLT(),not_design))

# Uncomment this command to see whether you set up the packer correctly
# This is useful to see what commands we're feeding in with our task factory
# print(tf.create_task_and_apply_taskoperations(pose_L99F))  

# # Create Packer
packer = pyrosetta.rosetta.protocols.minimization_packing.PackRotamersMover()
packer.task_factory(tf)

In [10]:
# Run repacking
pose_N501Y_repacked = pose_N501Y.clone()
pose_N501Y_repacked.pdb_info().name('N501' + one_letter_code + '_repacked')

packer.apply(pose_N501Y_repacked)

protocols.minimization_packing.PackRotamersMover: {0} [ WARNING ] undefined ScoreFunction -- creating a default one
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015_cart
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 1127 rotamers at 25 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
basic.thread_manager.RosettaThreadManager: {?} Creating a thread pool of 1 threads.
basic.thread_manager.RosettaThreadPool: {?} Launched 0 new threads.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).


In [11]:
N501Y_repacked_score = scorefxn(pose_N501Y_repacked)

print('\nWT score: ' + str(WT_score))
print('\nN501Y mutated score: ' + str(N501Y_score))
print('\nN501Y repacked score: ' + str(N501Y_repacked_score))

scaling_factor_for_ddG = 4
ddG_N501Y = N501Y_repacked_score - WT_score
print('\n\nDifference in score N501Y (repacked) - WT: ' + str(ddG_N501Y))
print('Computed ΔΔG of N501Y: ' + str(ddG_N501Y/scaling_factor_for_ddG))


WT score: -2940.9505527784117

N501Y mutated score: -2936.293605422784

N501Y repacked score: -2936.538822685187


Difference in score N501Y (repacked) - WT: 4.411730093224833
Computed ΔΔG of N501Y: 1.1029325233062082


N501Y_repacked has a score much closer to the WT score, but it is still worse by almost 100 REU (25 kcal/mol).  
Experiments suggest that N501Y should have a lower dG than WT, though.  
Let's see if I can visualize the repacking, or if pmm.apply(repacked_pose) still isn't communicating with my pymol session:

In [12]:
pmm.apply(pose_N501Y_repacked) # I'll have to troubleshoot this later

`util.cbaw N501Y_repacked`

`select N501Y_repacked_core, N501Y_repacked and br. (sc.  within 3 of (resi 501 and chain F and sc.))`

`util.cbac N501Y_repacked_core`

`util.cbas N501Y_repacked_core and resi 501 and chain F`

`show sticks, N501Y_repacked_core and not hydro`

`# show spheres, N501Y_repacked_core and (resi 84 or resi 99) and not (name O or name N)`

`set_view (\
     0.737348735,   -0.664588213,   -0.120811388,\
    -0.212751448,   -0.398235619,    0.892252982,\
    -0.641090214,   -0.632204175,   -0.435039639,\
     0.000094488,    0.001106668,  -85.698837280,\
    30.798748016,    4.481562614,    6.916054726,\
    49.036270142,  122.386810303,  -20.000000000 )`

`deselect`

In [13]:
### Some Notes on FastRelax and FastDesign

# FD and FR do the same things, except that FD can change residue type
# Simulated annealing in both

# If we want to run FastDesign instead, use a FastDesign object
# fd = pyrosetta.protocols.etc.etc.FastDesign
# fd.set_task_factory(tf)

# FastRelax 1) Repacks and 2) Minimizes
# Minimize structure after FD with FastRelax
# FastRelax takes a MoveMapFactory as well as a TaskFactory

Let's see how the WT energy terms compare to the N501Y_repacked ones:

In [14]:
energy_table_WT = pyrosetta.bindings.energies.residue_total_energies_array(pose_relaxed.energies())
energy_df_WT = pd.DataFrame(energy_table_WT)
totals_by_term_WT = energy_df_WT.sum(axis=0)


energy_table_N501Y_repacked = pyrosetta.bindings.energies.residue_total_energies_array(pose_N501Y_repacked.energies())
energy_df_N501Y_repacked = pd.DataFrame(energy_table_N501Y_repacked)
totals_by_term_N501Y_repacked = energy_df_N501Y_repacked.sum(axis=0)

totals_by_term_N501Y_repacked - totals_by_term_WT

fa_atr                 1.581669
fa_rep                -0.266753
fa_sol                 2.563585
fa_intra_rep          -0.193953
fa_intra_sol_xover4    0.046700
lk_ball_wtd            0.269701
fa_elec                1.458267
hbond_sr_bb            0.000000
hbond_lr_bb            0.000000
hbond_bb_sc            0.837584
hbond_sc               0.000000
dslf_fa13              0.000000
omega                  0.000000
fa_dun                -0.771438
p_aa_pp               -0.562633
yhh_planarity          0.000000
ref                   -0.805480
rama_prepro           -0.676571
cart_bonded           -0.428119
total_score            4.408239
dtype: float64

Still, fa_rep is responsible for the difference in REU.  If I knew how to fix the link between this notebook and my pymol session, then I'd probably see a steric clash between Y501 and the rest of the core.  
  
This shows that repacking was not enough to resolve the clash.

This suggests that the backbone as it currently is modeled is just not capable of incorporating the large phenyl group. We must therefore introduce changes to the backbone to see if we can create some space. We can do this by adding a minimization step to our repacking. At this point, we are just running a round of relax while restricting repacking to the vicinity around the mutation.

In [15]:
# Create movemap factory
mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
mmf.all_bb(setting=True)
mmf.all_bondangles(setting=True)
mmf.all_bondlengths(setting=True)
mmf.all_chi(setting=True)
mmf.all_jumps(setting=True)
mmf.set_cartesian(setting=True)

# Make fastrelax protocol
fr = pyrosetta.rosetta.protocols.relax.FastRelax(standard_repeats=1)
fr.set_scorefxn(scorefxn)
fr.set_task_factory(tf)
fr.set_movemap_factory(mmf)
fr.cartesian(True)
fr.min_type("lbfgs_armijo_nonmonotone")

pose_N501Y_relaxed = pose_N501Y.clone()
pose_N501Y_relaxed.pdb_info().name('N501' + one_letter_code + '_relaxed')

fr.apply(pose_N501Y_relaxed)

pose_N501Y_relaxed.dump_pdb('N501' + one_letter_code + '_relaxed.pdb')

protocols.relax.RelaxScriptManager: {0} Reading relax scripts list from database.
core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015_cart
protocols.relax.RelaxScriptManager: {0} Looking for MonomerRelax2019.txt
protocols.relax.RelaxScriptManager: {0} ================== Reading script file: /opt/anaconda3/envs/proteindesign/lib/python3.7/site-packages/pyrosetta/database/sampling/relax_scripts/MonomerRelax2019.txt ==================
protocols.relax.RelaxScriptManager: {0} repeat %%nrepeats%%
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 1.0
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.040
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.051
protocols.relax.RelaxScriptManager: {0} min 0.01
protocols.relax.RelaxScriptManager: {0} coord_cst_weight 0.5
protocols.relax.RelaxScriptManager: {0} scale:fa_rep 0.265
protocols.relax.RelaxScriptManager: {0} repack
protocols.relax.RelaxScriptManager: {0} scale:fa_rep

protocols.relax.FastRelax: {0} CMD: min  -3290.39  0.439234  0.439234  0.31955
protocols.relax.FastRelax: {0} CMD: coord_cst_weight  -3290.39  0.439234  0.439234  0.31955
protocols.relax.FastRelax: {0} CMD: scale:fa_rep  -2825.99  0.439234  0.439234  0.55
core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 1017 rotamers at 23 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).
protocols.relax.FastRelax: {0} CMD: repack  -2825.99  0.439234  0.439234  0.55
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch connection angle: residue 324 BRANCH 1
core.pose.util: {0} [ WARNING ] Unable to find atom_tree atom for this Rosetta branch 

True

In [16]:
energy_table_WT = pyrosetta.bindings.energies.residue_total_energies_array(pose_relaxed.energies())
energy_df_WT = pd.DataFrame(energy_table_WT)
totals_by_term_WT = energy_df_WT.sum(axis=0)


energy_table_N501Y_relaxed = pyrosetta.bindings.energies.residue_total_energies_array(pose_N501Y_relaxed.energies())
energy_df_N501Y_relaxed = pd.DataFrame(energy_table_N501Y_relaxed)
totals_by_term_N501Y_relaxed = energy_df_N501Y_relaxed.sum(axis=0)

totals_by_term_N501Y_relaxed - totals_by_term_WT

fa_atr                 0.312541
fa_rep                 7.643323
fa_sol                 3.340095
fa_intra_rep          -9.762729
fa_intra_sol_xover4    0.085646
lk_ball_wtd           -0.377927
fa_elec                0.360330
hbond_sr_bb            0.000000
hbond_lr_bb            0.000000
hbond_bb_sc            0.196064
hbond_sc              -0.211597
dslf_fa13             -0.004759
omega                  0.732051
fa_dun                -5.262142
p_aa_pp               -3.594440
yhh_planarity          0.368446
ref                   -0.805480
rama_prepro           -1.014478
cart_bonded            5.407313
total_score            3.978815
dtype: float64

In [17]:
N501Y_relaxed_score = scorefxn(pose_N501Y_relaxed)
print('\nWT score: ' + str(WT_score))
print('\nN501Y mutated score: ' + str(N501Y_score))
print('\nN501Y repacked score: ' + str(N501Y_repacked_score))
print('\nN501Y relaxed score: ' + str(N501Y_relaxed_score))

ddG_N501Y = N501Y_relaxed_score - WT_score
print('\n\nDifference in score N501Y (relaxed) - WT: ' + str(ddG_N501Y))
print('Computed ΔΔG of N501Y: ' + str(ddG_N501Y/scaling_factor_for_ddG))


WT score: -2940.9505527784117

N501Y mutated score: -2936.293605422784

N501Y repacked score: -2936.538822685187

N501Y relaxed score: -2936.2921457999364


Difference in score N501Y (relaxed) - WT: 4.658406978475341
Computed ΔΔG of N501Y: 1.1646017446188353


#### Now, our ddG of N501Y is only 1.47 kcal/mol. 
We may be able to get closer to the "true" native state of N501Y with additional and/or longer rounds of relax, as the increased sampling of backbones may find one even better suited for the mutation.

# Next, do this same pipeline for every other natural mutation

Let's look at our new structure.

`util.cbaw N501Y_relaxed`

`select N501Y_relaxed_core, N501Y_relaxed and br. (sc.  within 3 of (resi 501 and chain F and sc.))`

`util.cbac N501Y_relaxed_core`

`util.cbas N501Y_relaxed_core and resi 501 and chain F`

`show sticks, N501Y_relaxed_core and not hydro`

`# show spheres, N501Y_relaxed_core and (resi 84 or resi 99) and not (name O or name N)`

`set_view (\
     0.737348735,   -0.664588213,   -0.120811388,\
    -0.212751448,   -0.398235619,    0.892252982,\
    -0.641090214,   -0.632204175,   -0.435039639,\
     0.000094488,    0.001106668,  -85.698837280,\
    30.798748016,    4.481562614,    6.916054726,\
    49.036270142,  122.386810303,  -20.000000000 )`

`deselect`

# Results:

    - WT score: -2940.9505527784117

    - N501Y (TYR):
    
    N501Y mutated score: -2516.6114822687923

    N501Y repacked score: -2863.518380580556

    N501Y relaxed score: -2937.989411368924
    
    - N501F (PHE):
    
    N501F mutated score: -2604.0158374597745

    N501F repacked score: -2803.2183208609313

    N501F relaxed score: -2934.2859407163996
    
    - N501W (TRP):
    
    N501W mutated score: -2112.8508010677433

    N501W repacked score: -2683.745897282113

    N501W relaxed score: -2933.9211385087747
    
    - N501V (VAL):
    
    N501V mutated score: -2871.507017266515

    N501V repacked score: -2889.545018445958

    N501V relaxed score: -2938.723241358529
    
    - N501M (MET):
    
    N501M mutated score: -2812.4625176081736

    N501M repacked score: -2920.2388943742326

    N501M relaxed score: -2943.9243932016493
    
    - N501G (GLY):
    
    N501G mutated score: -2937.9700708917426

    N501G repacked score: -2937.9700708917426

    N501G relaxed score: -2937.4887851571743
    
    - N501P (PRO):
    
    N501P mutated score: -2789.7197845537457

    N501P repacked score: -2796.895715567556

    N501P relaxed score: -2932.514215871043
    
    - N501L (LEU):
    
    N501L mutated score: -2842.0661521941593

    N501L repacked score: -2925.555118719668

    N501L relaxed score: -2944.7714219671925
    
    - N501I (ILE):
    
    N501I mutated score: -2826.5488432498755

    N501I repacked score: -2841.8804037426917

    N501I relaxed score: -2937.4648232005916
    
    - N501A (ALA):
    
    N501A mutated score: -2941.8618565280094

    N501A repacked score: -2941.8618565280094

    N501A relaxed score: -2939.3242087955628
    
    - N501D (ASP):
    
    N501D mutated score: -2936.293605422784

    N501D repacked score: -2936.538822685187

    N501D relaxed score: -2936.2921457999364
    
    
    
    
    


### Rerun the above mutations, now that I'm writing out N501X_relaxed.pdb

- MET (M), VAL (V), TRP (W), PHE (F), TYR (Y)
- GLY (G), PRO (P), LEU (L), ILE (I), ALA (A)
- ASP (D),

In [18]:
pmm.apply(pose_N501Y_relaxed)

## wt_vs_mut to compare structures of mutants to wt/other mutants

PyMol Cmd: wt_vs_mut 'name of pymol object 1', 'name of pymol object 2'  
ex/ wt_vs_mut 6M17_relaxed, N501M_relaxed

## The Rosetta Energies I've been outputting are scores for the complex, but the dG of binding is really the difference between this score and the scores of the individual chains when unbound.

ddG would then be the difference between the dG of binding of the WT and the mutant

In [ ]:
## From Chris Mathy: Function to measure interface E
## Separates two chains far apart, then recalculates Rosetta E

# Leave pack_input and pack_separated as False
# Might just introduce inconsistencies

# If dump_pdb_filename='.pdb', then the pdb file it spits 

def get_interface_energy(pose, scorefxn, interface,
                  pack_input=False, pack_separated=False,
                  dump_pdb_filename=None):
    '''
    pose: the complex to be scored
    scorefxn: a rosetta score function
    interface: str listing the chains forming the interface, i.e. A_B
    pack_input: bool, whether to re-pack the input structure
    pack_separated: bool, whether to re-pack the structures after seperation
    dump_pdb: str, path to file to dump the pdb with the interface analyzer scores in the file. Defaults to NULL, no dump.
    '''
    p = pose.clone()
    ia_mover = pyrosetta.rosetta.protocols.analysis.InterfaceAnalyzerMover()
    ia_mover.set_scorefunction(scorefxn)
    ia_mover.set_interface(interface)
    ia_mover.set_pack_input(pack_input)
    ia_mover.set_pack_separated(pack_separated)
    ia_mover.apply(p)
    ia_mover.add_score_info_to_pose(p)
    if (dump_pdb_filename):
        p.dump_pdb(dump_pdb_filename)
    return ia_mover.get_separated_interface_energy()

In [ ]:
# pose = pose_from_pdb('input.pdb')
# scorefxn = create_score_function('ref2015_cart')
# interface_energy = get_interface_energy(pose=pose, scorefxn=scorefxn, interface='D_F', dump_pdb_filename='test.pdb')